In [117]:
# ============================================================================
# BLOCK 1: IMPORTS AND SETUP
# ============================================================================
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Directory for processed outputs
processed_dir = Path("processed")
processed_dir.mkdir(exist_ok=True, parents=True)

In [118]:
# ============================================================================
# BLOCK 2: UTILITY FUNCTIONS
# ============================================================================
def load_all_sheets(excel_path: str) -> pd.DataFrame:
    """
    Loads all sheets from an Excel file into one combined DataFrame.
    Adds sheet name and file info for traceability.
    """
    all_dfs = []
    try:
        xls = pd.ExcelFile(excel_path)
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(excel_path, sheet_name=sheet_name)
            df["source_sheet"] = sheet_name
            all_dfs.append(df)
    except Exception as e:
        print(f" Could not read {excel_path}: {e}")
        return pd.DataFrame()
    combined = pd.concat(all_dfs, ignore_index=True)
    combined["source_file"] = Path(excel_path).stem
    return combined

def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("(", "", regex=False)
        .str.replace(")", "", regex=False)
    )
    return df


In [119]:
# ============================================================================
# BLOCK 3: LOAD MONTHLY SALES DATA
# ============================================================================
monthly_files = {
    "2025-05": "May_Data_Matrix (1).xlsx",
    "2025-06": "June_Data_Matrix.xlsx",
    "2025-07": "July_Data_Matrix (1).xlsx",
    "2025-08": "August_Data_Matrix (1).xlsx",
    "2025-09": "September_Data_Matrix.xlsx",
    "2025-10": "October_Data_Matrix_20251103_214000.xlsx",
}

monthly_frames = []

for period, file in monthly_files.items():
    df = load_all_sheets(file)
    if df.empty:
        continue
    df = clean_column_names(df)
    
    # FIX: Handle all column variations properly
    # Unify 'category' -> 'group'
    if "category" in df.columns and "group" not in df.columns:
        df = df.rename(columns={"category": "group"})
    
    # Unify 'item_name' -> 'item'
    if "item_name" in df.columns and "item" not in df.columns:
        df = df.rename(columns={"item_name": "item"})
    
    # FIX: Create unified 'group' column from available columns
    if "group" not in df.columns:
        df["group"] = None
    df["group"] = df["group"].fillna(df.get("category")).fillna(df.get("item"))
    
    df["period"] = period
    monthly_frames.append(df)

# Combine all months
monthly = pd.concat(monthly_frames, ignore_index=True)

# FIX: Convert BOTH count AND amount (not just count)
for col in ["count", "amount"]:
    if col in monthly.columns:
        # Remove dollar signs and commas from amount if present
        if col == "amount" and monthly[col].dtype == 'object':
            monthly[col] = monthly[col].str.replace('$', '', regex=False).str.replace(',', '', regex=False)
        monthly[col] = pd.to_numeric(monthly[col], errors="coerce")

# Create a clean 'item' column for merging with BOM
if "item" not in monthly.columns:
    monthly["item"] = np.nan

monthly["item_norm"] = monthly["item"].fillna(monthly["group"]).str.strip().str.lower()

print(f"Combined monthly shape: {monthly.shape}")
monthly.head(5)

Combined monthly shape: (795, 11)


,source_page,source_table,group,count,amount,source_sheet,category,item,source_file,period,item_norm
0,1,1,Lunch Menu,269.0,3783.26,data 1,NaN,NaN,May_Data_Matrix (1),2025-05,lunch menu
1,1,1,Open Food,2.0,4.52,data 1,NaN,NaN,May_Data_Matrix (1),2025-05,open food
2,1,1,All Day Menu,NaN,61296.15,data 1,NaN,NaN,May_Data_Matrix (1),2025-05,all day menu
3,1,1,Fried Chicken,787.0,7935.26,data 2,Fried Chicken,NaN,May_Data_Matrix (1),2025-05,fried chicken
4,1,1,Combo Items,0.0,0.00,data 2,Combo Items,NaN,May_Data_Matrix (1),2025-05,combo items


In [120]:

# ============================================================================
# BLOCK 4: AGGREGATE SALES DATA
# ============================================================================
monthly_grouped = (
    monthly.groupby(["period", "group"], dropna=False)[["count", "amount"]]
    .sum()
    .reset_index()
    .sort_values(["period", "group"])
)
print(f"Aggregated monthly summary: {monthly_grouped.shape}")
display(monthly_grouped.head(10))

Aggregated monthly summary: (760, 4)


,period,group,count,amount
0,2025-05,Additonal,175.0,358.48
1,2025-05,All Day Menu,0.0,61296.15
2,2025-05,Appetizer,0.0,6217.05
3,2025-05,BF chicken cutlet,10.0,65.93
4,2025-05,BF chicken cutlet combo,20.0,242.53
5,2025-05,Beef Fried Rice,28.0,377.50
6,2025-05,Beef Ramen,339.0,5045.65
7,2025-05,Beef Rice Noodle Soup,147.0,2316.04
8,2025-05,Beef Tossed Ramen,468.0,6921.26
9,2025-05,Beef Tossed Rice Noodle,195.0,3102.44


In [121]:
# ============================================================================
# BLOCK 5: LOAD INGREDIENT AND SHIPMENT DATA
# ============================================================================
def load_csv_or_excel(path: str) -> pd.DataFrame:
    """
    Loads CSV or Excel with all sheets if available.
    Returns a single unified DataFrame.
    """
    if path.endswith(".csv"):
        return pd.read_csv(path)
    elif path.endswith(".xlsx"):
        return load_all_sheets(path)
    else:
        raise ValueError("Unsupported file type: must be .csv or .xlsx")

# Load raw files
ingredient_df = load_csv_or_excel("MSY Data - Ingredient.csv")
shipment_df = load_csv_or_excel("MSY Data - Shipment.csv")

# Clean names
ingredient_df = clean_column_names(ingredient_df)
shipment_df = clean_column_names(shipment_df)

print("Ingredient Columns:", ingredient_df.columns.tolist())
print("Shipment Columns:", shipment_df.columns.tolist())


Ingredient Columns: ['item_name', 'braised_beef_used_g', 'braised_chickeng', 'braised_porkg', 'eggcount', 'riceg', 'ramen_count', 'rice_noodlesg', 'chicken_thigh_pcs', 'chicken_wings_pcs', 'flour_g', 'pickle_cabbage', 'green_onion', 'cilantro', 'white_onion', 'peasg', 'carrotg', 'boychoyg', 'tapioca_starch']
Shipment Columns: ['ingredient', 'quantity_per_shipment', 'unit_of_shipment', 'number_of_shipments', 'frequency']


In [122]:
# ============================================================================
# BLOCK 6: TRANSFORM BOM TO LONG FORMAT
# ============================================================================
idf = ingredient_df.copy()
if "item_name" not in idf.columns:
    potential_name_cols = [c for c in idf.columns if "item" in c]
    if potential_name_cols:
        idf = idf.rename(columns={potential_name_cols[0]: "item_name"})
    else:
        raise KeyError(" No 'item_name' or similar column found in ingredient dataset.")

ingredient_cols = [c for c in idf.columns if c != "item_name"]

def parse_unit(col):
    if col.endswith("_g") or col.endswith("g"):
        return "g"
    if col.endswith("_count") or col.endswith("count"):
        return "count"
    if col.endswith("_pcs") or col.endswith("pcs"):
        return "pcs"
    return "units"

def normalize_ingredient_name(col):
    name = col
    name = re.sub(r"(_)?g$", "", name)
    name = re.sub(r"(_)?count$", "", name)
    name = re.sub(r"(_)?pcs$", "", name)
    name = name.replace("_", " ").strip()
    return name

long_rows = []
for _, row in idf.iterrows():
    item = row.get("item_name")
    for col in ingredient_cols:
        try:
            num = float(row[col])
        except (TypeError, ValueError):
            continue
        if pd.isna(num) or num == 0:
            continue
        long_rows.append({
            "item_name": item,
            "ingredient": normalize_ingredient_name(col),
            "quantity_per_item": num,
            "unit": parse_unit(col),
        })

ingredient_long = pd.DataFrame(long_rows).sort_values(["item_name", "ingredient"])
ingredient_long["ingredient_norm"] = ingredient_long["ingredient"].str.strip().str.lower()
ingredient_long["item_norm"] = ingredient_long["item_name"].str.strip().str.lower()

print("BOM Long Format Created:", ingredient_long.shape)
display(ingredient_long.head(10))


BOM Long Format Created: (98, 6)


,item_name,ingredient,quantity_per_item,unit,ingredient_norm,item_norm
12,Beef Fried Rice,braised beef used,100.0,g,braised beef used,beef fried rice
18,Beef Fried Rice,carrot,10.0,g,carrot,beef fried rice
13,Beef Fried Rice,egg,1.0,count,egg,beef fried rice
15,Beef Fried Rice,green onion,20.0,units,green onion,beef fried rice
17,Beef Fried Rice,peas,10.0,g,peas,beef fried rice
14,Beef Fried Rice,rice,350.0,g,rice,beef fried rice
16,Beef Fried Rice,white onion,20.0,units,white onion,beef fried rice
7,Beef Ramen,braised beef used,140.0,g,braised beef used,beef ramen
11,Beef Ramen,cilantro,20.0,units,cilantro,beef ramen
8,Beef Ramen,egg,0.5,count,egg,beef ramen


In [123]:
# ============================================================================
# BLOCK 7: NORMALIZE SHIPMENT DATA WITH UNIT CONVERSION
# ============================================================================
ship = shipment_df.copy()

# Ingredient mapping
INGREDIENT_MAP = {
    'beef': 'braised beef used',
    'chicken': 'braised chicken',
    'ramen': 'ramen',
    'rice noodles': 'rice noodles',
    'flour': 'flour',
    'tapioca starch': 'tapioca starch',
    'rice': 'rice',
    'green onion': 'green onion',
    'white onion': 'white onion',
    'cilantro': 'cilantro',
    'egg': 'egg',
    'bokchoy': 'bokchoy',
    'chicken wings': 'chicken wings'
}

# Normalize ingredient column
if "ingredient" not in ship.columns:
    raise KeyError("'ingredient' column not found in shipment dataset.")

ship["ingredient_norm"] = ship["ingredient"].str.strip().str.lower()

# FIX: Handle 'peas + carrot' by splitting into separate rows
peas_carrot_mask = ship["ingredient_norm"] == "peas + carrot"
if peas_carrot_mask.any():
    peas_carrot_rows = ship[peas_carrot_mask].copy()
    
    # Create peas row
    peas_row = peas_carrot_rows.copy()
    peas_row["ingredient_norm"] = "peas"
    peas_row["quantity_per_shipment"] = peas_row["quantity_per_shipment"] / 2
    
    # Create carrot row
    carrot_row = peas_carrot_rows.copy()
    carrot_row["ingredient_norm"] = "carrot"
    carrot_row["quantity_per_shipment"] = carrot_row["quantity_per_shipment"] / 2
    
    # Remove original and add split rows
    ship = pd.concat([
        ship[~peas_carrot_mask],
        peas_row,
        carrot_row
    ], ignore_index=True)

# Apply the mapping
ship["ingredient_norm"] = ship["ingredient_norm"].map(INGREDIENT_MAP).fillna(ship["ingredient_norm"])

# Convert number columns
for col in ["quantity_per_shipment", "number_of_shipments"]:
    if col in ship.columns:
        ship[col] = pd.to_numeric(ship[col], errors="coerce")

# FIX: CRITICAL - Convert quantities to grams based on unit
def convert_to_grams(row):
    """Convert shipment quantities to grams."""
    qty = row['quantity_per_shipment']
    unit = str(row.get('unit_of_shipment', '')).lower()
    
    if 'lb' in unit:
        return qty * 453.592  # pounds to grams
    elif 'egg' in unit:
        return qty * 50  # assume 50g per egg
    elif 'onion' in unit or 'whole' in unit:
        return qty * 150  # assume 150g per onion
    elif 'roll' in unit:
        return qty * 100  # assume 100g per roll
    elif 'piece' in unit or 'pcs' in unit:
        return qty * 100  # assume 100g per piece
    else:
        return qty  # already in grams

ship['quantity_in_grams'] = ship.apply(convert_to_grams, axis=1)

print("Shipment Data Normalized (with Unit Conversion):", ship.shape)
display(ship.head(10))


Shipment Data Normalized (with Unit Conversion): (15, 7)


,ingredient,quantity_per_shipment,unit_of_shipment,number_of_shipments,frequency,ingredient_norm,quantity_in_grams
0,Beef,40.0,lbs,3,weekly,braised beef used,18143.68
1,Chicken,40.0,lbs,2,weekly,braised chicken,18143.68
2,Ramen,50.0,rolls,15,biweekly,ramen,5000.00
3,Rice Noodles,50.0,lbs,2,monthly,rice noodles,22679.60
4,Flour,50.0,lbs,2,monthly,flour,22679.60
5,Tapioca Starch,25.0,lbs,1,monthly,tapioca starch,11339.80
6,Rice,50.0,lbs,2,Biweekly,rice,22679.60
7,Green Onion,20.0,lbs,2,weekly,green onion,9071.84
8,White Onion,80.0,whole onion,4,weekly,white onion,12000.00
9,Cilantro,5.0,lbs,2,weekly,cilantro,2267.96


In [124]:
# ============================================================================
# BLOCK 8: SAVE PROCESSED DATA
# ============================================================================
sales_out = processed_dir / "sales_category_monthly.csv"
bom_out = processed_dir / "ingredient_bom_long.csv"
ship_out = processed_dir / "shipments_clean.csv"
sales_item_out = processed_dir / "sales_item_monthly.csv"

monthly_grouped.to_csv(sales_out, index=False)
ingredient_long.to_csv(bom_out, index=False)
ship.to_csv(ship_out, index=False)
monthly.to_csv(sales_item_out, index=False)

print("Saved Processed Data:")
print("-", sales_out)
print("-", bom_out)
print("-", ship_out)
print("-", sales_item_out)

Saved Processed Data:
- processed/sales_category_monthly.csv
- processed/ingredient_bom_long.csv
- processed/shipments_clean.csv
- processed/sales_item_monthly.csv


In [125]:
# ============================================================================
# BLOCK 9: VERIFY MERGE PREVIEW
# ============================================================================
print("Ingredients in BOM:", ingredient_long["ingredient_norm"].nunique())
print("Ingredients in Shipments:", ship["ingredient_norm"].nunique())

merged_preview = ingredient_long.merge(
    ship, 
    on="ingredient_norm", 
    how="left",
    suffixes=('_bom', '_shipment')
)
display(merged_preview.sample(5))


Ingredients in BOM: 18
Ingredients in Shipments: 15


,item_name,ingredient_bom,quantity_per_item,unit,ingredient_norm,item_norm,ingredient_shipment,quantity_per_shipment,unit_of_shipment,number_of_shipments,frequency,quantity_in_grams
50,Chicken Rice Noodle Soup,egg,0.5,count,egg,chicken rice noodle soup,Egg,120.0,eggs,5.0,weekly,6000.00
31,Chicken Cutlet,chicken thigh,1.0,pcs,chicken thigh,chicken cutlet,NaN,NaN,NaN,NaN,NaN,NaN
84,Pork Rice Noodle Soup,green onion,20.0,units,green onion,pork rice noodle soup,Green Onion,20.0,lbs,2.0,weekly,9071.84
94,Pork Tossed Rice Noodles,egg,0.5,count,egg,pork tossed rice noodles,Egg,120.0,eggs,5.0,weekly,6000.00
38,Chicken Fried Rice,peas,10.0,g,peas,chicken fried rice,Peas + Carrot,20.0,lbs,3.0,weekly,9071.84


In [126]:
# ============================================================================
# BLOCK 10: GENERATE KPI SUMMARY
# ============================================================================
import pandas as pd

sales = pd.read_csv("processed/sales_category_monthly.csv")

# FIX: Compute KPIs for BOTH count AND amount
kpi_counts = (
    sales.groupby("period")[["count", "amount"]]
    .sum()
    .reset_index()
    .sort_values("period")
)

# Month-over-month % change for both metrics
kpi_counts["count_mom_growth_%"] = kpi_counts["count"].pct_change() * 100
kpi_counts["amount_mom_growth_%"] = kpi_counts["amount"].pct_change() * 100

print("KPI Summary Ready")
display(kpi_counts)


KPI Summary Ready


,period,count,amount,count_mom_growth_%,amount_mom_growth_%
0,2025-05,11204.0,195251.76,NaN,NaN
1,2025-06,8391.0,127581.98,-25.107105,-34.657706
2,2025-07,9121.0,138285.61,8.699797,8.389610
3,2025-08,12169.0,195651.62,33.417388,41.483716
4,2025-09,13959.0,225709.82,14.709508,15.363123
5,2025-10,13732.0,219622.04,-1.626191,-2.697171


In [127]:
# ============================================================================
# BLOCK 11: TOP/BOTTOM CATEGORY ANALYSIS 
# ============================================================================
top_bottom = (
    sales.groupby(["period", "group"])[["count", "amount"]]
    .sum()
    .reset_index()
)

# Top 5 categories by amount (revenue)
top5 = top_bottom.sort_values(["period", "amount"], ascending=[True, False]).groupby("period").head(5)

# Bottom 5 categories by amount
bottom5 = top_bottom.sort_values(["period", "amount"], ascending=[True, True]).groupby("period").head(5)

print("Top & Bottom Category Tables Ready")
display(top5.head(10))


Top & Bottom Category Tables Ready


,period,group,count,amount
1,2025-05,All Day Menu,0.0,61296.15
95,2025-05,Tossed Ramen,941.0,13539.42
73,2025-05,Ramen,822.0,12127.38
39,2025-05,Fried Chicken,787.0,7935.26
78,2025-05,Rice Noodle,541.0,7724.71
114,2025-06,All Day Menu,0.0,39433.25
220,2025-06,Tossed Ramen,510.0,7287.69
194,2025-06,Ramen,485.0,7116.70
166,2025-06,Lunch Special,444.0,6178.16
152,2025-06,Fried Chicken,551.0,5725.04


In [128]:

# ============================================================================
# BLOCK 12: INGREDIENT DEMAND CALCULATION
# ============================================================================
sales_items = pd.read_csv("processed/sales_item_monthly.csv")
bom = pd.read_csv("processed/ingredient_bom_long.csv")

# Sum sales by normalized item name and period
item_sales = sales_items.groupby(["period", "item_norm"])["count"].sum().reset_index()

# Merge sales with BOM
demand = item_sales.merge(bom, on="item_norm", how="left")

# Calculate ingredient demand
demand["ingredient_demand"] = demand["count"] * demand["quantity_per_item"]

# Aggregate total demand for each ingredient by month
ingredient_forecast_df = (
    demand.groupby(["period", "ingredient_norm", "unit"])["ingredient_demand"]
    .sum()
    .reset_index()
    .sort_values(by=["period", "ingredient_demand"], ascending=[True, False])
)

ingredient_forecast_df = ingredient_forecast_df.rename(
    columns={"ingredient_norm": "ingredient", "ingredient_demand": "total_usage"}
)

print("Ingredient Demand Forecast Ready")
display(ingredient_forecast_df.head(10))


Ingredient Demand Forecast Ready


,period,ingredient,unit,total_usage
12,2025-05,rice noodles,g,171000.0
1,2025-05,braised beef used,g,136360.0
2,2025-05,braised chicken,g,116220.0
3,2025-05,braised pork,g,66520.0
0,2025-05,boychoy,g,63650.0
9,2025-05,pickle cabbage,units,52300.0
7,2025-05,green onion,units,46300.0
5,2025-05,cilantro,units,43800.0
11,2025-05,rice,g,43750.0
13,2025-05,white onion,units,2500.0


In [129]:
# ============================================================================
# BLOCK 13: SHIPMENT RELIABILITY SUMMARY 
# ============================================================================
shipment = pd.read_csv("processed/shipments_clean.csv")

# FIX: Use quantity_in_grams for calculations
shipment["avg_quantity_per_shipment_grams"] = shipment["quantity_in_grams"] / shipment["number_of_shipments"]

# Map frequency to weeks
freq_map = {'weekly': 1, 'biweekly': 2, 'monthly': 4}
shipment["weeks_between_shipments"] = shipment["frequency"].str.lower().map(freq_map).fillna(4)

# Clean final table - MUST include 'ingredient_norm' for merging
ship_summary = shipment[[
    "ingredient",
    "ingredient_norm",
    "number_of_shipments",
    "avg_quantity_per_shipment_grams",  # FIX: Use grams
    "weeks_between_shipments"
]]

print("Shipment Reliability Summary Ready")
display(ship_summary.head(10))


Shipment Reliability Summary Ready


,ingredient,ingredient_norm,number_of_shipments,avg_quantity_per_shipment_grams,weeks_between_shipments
0,Beef,braised beef used,3,6047.893333,1
1,Chicken,braised chicken,2,9071.840000,1
2,Ramen,ramen,15,333.333333,2
3,Rice Noodles,rice noodles,2,11339.800000,4
4,Flour,flour,2,11339.800000,4
5,Tapioca Starch,tapioca starch,1,11339.800000,4
6,Rice,rice,2,11339.800000,2
7,Green Onion,green onion,2,4535.920000,1
8,White Onion,white onion,4,3000.000000,1
9,Cilantro,cilantro,2,1133.980000,1


In [130]:
# ============================================================================
# BLOCK 14: REORDER RECOMMENDATIONS
# ============================================================================
# Calculate average usage across all months
reorder = ingredient_forecast_df.groupby("ingredient")["total_usage"].mean().reset_index()

# FIX: Merge with proper column naming (no _x, _y)
reorder = reorder.merge(
    ship_summary, 
    left_on="ingredient",
    right_on="ingredient_norm",
    how="left",
    suffixes=('', '_dup')  # Prevent _x, _y
)

# Drop duplicate columns
reorder = reorder.drop(columns=[c for c in reorder.columns if c.endswith('_dup')])

# FIX: Calculate with grams (both usage and shipment in same units)
reorder["weekly_usage_estimate"] = reorder["total_usage"] / 4.33

# FIX: Use grams for both numerator and denominator
reorder["days_until_depletion"] = (
    reorder["avg_quantity_per_shipment_grams"] / reorder["weekly_usage_estimate"]
) * 7

# Generate reorder alerts with better thresholds
def get_alert(days):
    if pd.isna(days):
        return "🔵 Unknown (No Shipment Data)"
    elif days < 7:
        return "🔴 Critical - Reorder Now"
    elif days < 14:
        return "🟠 Reorder Soon"
    else:
        return "🟢 Sufficient"

reorder["reorder_alert"] = reorder["days_until_depletion"].apply(get_alert)

print("Reorder Suggestion Table Ready")
print("\nAlert Distribution:")
print(reorder['reorder_alert'].value_counts())
display(reorder.head(15))


Reorder Suggestion Table Ready

Alert Distribution:
reorder_alert
🔴 Critical - Reorder Now        6
🟢 Sufficient                    4
🔵 Unknown (No Shipment Data)    3
🟠 Reorder Soon                  1
Name: count, dtype: int64


,ingredient,total_usage,ingredient_norm,number_of_shipments,avg_quantity_per_shipment_grams,weeks_between_shipments,weekly_usage_estimate,days_until_depletion,reorder_alert
0,boychoy,46791.666667,NaN,NaN,NaN,NaN,10806.389530,NaN,🔵 Unknown (No Shipment Data)
1,braised beef used,101590.000000,braised beef used,3.0,6047.893333,1.0,23461.893764,1.804426,🔴 Critical - Reorder Now
2,braised chicken,86810.000000,braised chicken,2.0,9071.840000,1.0,20048.498845,3.167463,🔴 Critical - Reorder Now
3,braised pork,54680.000000,NaN,NaN,NaN,NaN,12628.175520,NaN,🔵 Unknown (No Shipment Data)
4,carrot,1786.666667,carrot,3.0,3023.946667,1.0,412.625096,51.299901,🟢 Sufficient
5,cilantro,32173.333333,cilantro,2.0,1133.980000,1.0,7430.331024,1.068305,🔴 Critical - Reorder Now
6,egg,983.000000,egg,5.0,1200.000000,1.0,227.020785,37.001017,🟢 Sufficient
7,green onion,35746.666667,green onion,2.0,4535.920000,1.0,8255.581216,3.846057,🔴 Critical - Reorder Now
8,peas,1786.666667,peas,3.0,3023.946667,1.0,412.625096,51.299901,🟢 Sufficient
9,pickle cabbage,31916.666667,NaN,NaN,NaN,NaN,7371.054657,NaN,🔵 Unknown (No Shipment Data)


In [131]:
# ============================================================================
# BLOCK 15: SAVE ALL ANALYTICS
# ============================================================================
kpi_counts.to_csv("processed/analytics_kpi_summary.csv", index=False)
top5.to_csv("processed/analytics_top5_categories.csv", index=False)
bottom5.to_csv("processed/analytics_bottom5_categories.csv", index=False)
ingredient_forecast_df.to_csv("processed/analytics_forecast_ready.csv", index=False)
ship_summary.to_csv("processed/analytics_shipment_summary.csv", index=False)
reorder.to_csv("processed/analytics_reorder_table.csv", index=False)

print("All analytics tables saved to /processed/")

All analytics tables saved to /processed/


In [132]:
# ============================================================================
# BLOCK 16: DEMAND FORECASTING - 3 MONTH PROJECTION
# ============================================================================

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load historical demand data
forecast_data = pd.read_csv("processed/analytics_forecast_ready.csv")
forecast_data['period'] = pd.to_datetime(forecast_data['period'])

# Function to calculate trend-based forecast
def forecast_ingredient_demand(ingredient_df, periods_ahead=3):
    """
    Forecast demand for next N months using linear regression trend
    
    Parameters:
    - ingredient_df: DataFrame with historical data for one ingredient
    - periods_ahead: Number of months to forecast (default: 3)
    
    Returns:
    - DataFrame with forecasted demand
    """
    if len(ingredient_df) < 2:
        # Not enough data for trend - use last known value
        last_value = ingredient_df['total_usage'].iloc[-1]
        return pd.DataFrame({
            'forecast_value': [last_value] * periods_ahead,
            'trend_strength': ['insufficient_data'] * periods_ahead
        })
    
    # Prepare data for linear regression
    ingredient_df = ingredient_df.sort_values('period').reset_index(drop=True)
    x = np.arange(len(ingredient_df))
    y = ingredient_df['total_usage'].values
    
    # Calculate linear regression manually
    n = len(x)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    # Calculate slope and intercept
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sum((x - x_mean) ** 2)
    
    if denominator == 0:
        slope = 0
        intercept = y_mean
    else:
        slope = numerator / denominator
        intercept = y_mean - slope * x_mean
    
    # Calculate R-squared for trend strength
    y_pred = slope * x + intercept
    ss_res = np.sum((y - y_pred) ** 2)
    ss_tot = np.sum((y - y_mean) ** 2)
    
    if ss_tot == 0:
        r_squared = 0
    else:
        r_squared = 1 - (ss_res / ss_tot)
    
    # Classify trend strength
    if r_squared > 0.7:
        trend = 'strong'
    elif r_squared > 0.4:
        trend = 'moderate'
    else:
        trend = 'weak'
    
    # Generate forecasts
    future_x = np.arange(len(x), len(x) + periods_ahead)
    forecasted_values = slope * future_x + intercept
    
    # Ensure non-negative forecasts
    forecasted_values = np.maximum(forecasted_values, 0)
    
    return pd.DataFrame({
        'forecast_value': forecasted_values,
        'trend_strength': [trend] * periods_ahead,
        'r_squared': [r_squared] * periods_ahead,
        'slope': [slope] * periods_ahead
    })

# Generate forecasts for all ingredients
print("Generating 3-month demand forecasts...")

forecasts = []
last_period = forecast_data['period'].max()

for ingredient in forecast_data['ingredient'].unique():
    ingredient_data = forecast_data[forecast_data['ingredient'] == ingredient].copy()
    unit = ingredient_data['unit'].iloc[0]
    
    # Generate forecast
    forecast_result = forecast_ingredient_demand(ingredient_data, periods_ahead=3)
    
    # Create future periods (next 3 months)
    future_periods = []
    for i in range(1, 4):
        # Add months properly
        future_date = last_period + pd.DateOffset(months=i)
        future_periods.append(future_date)
    
    # Combine with forecast results
    for i, period in enumerate(future_periods):
        forecasts.append({
            'ingredient': ingredient,
            'period': period,
            'forecasted_usage': forecast_result['forecast_value'].iloc[i],
            'unit': unit,
            'trend_strength': forecast_result['trend_strength'].iloc[i],
            'r_squared': forecast_result['r_squared'].iloc[i],
            'slope': forecast_result['slope'].iloc[i],
            'forecast_type': 'linear_trend'
        })

# Create forecast DataFrame
forecast_3months = pd.DataFrame(forecasts)

print(f"Generated forecasts for {forecast_3months['ingredient'].nunique()} ingredients")
print(f"Forecast periods: {forecast_3months['period'].min()} to {forecast_3months['period'].max()}")

display(forecast_3months.head(15))

Generating 3-month demand forecasts...
Generated forecasts for 14 ingredients
Forecast periods: 2025-11-01 00:00:00 to 2026-01-01 00:00:00


,ingredient,period,forecasted_usage,unit,trend_strength,r_squared,slope,forecast_type
0,rice noodles,2025-11-01,112020.000000,g,weak,0.098374,-4208.571429,linear_trend
1,rice noodles,2025-12-01,107811.428571,g,weak,0.098374,-4208.571429,linear_trend
2,rice noodles,2026-01-01,103602.857143,g,weak,0.098374,-4208.571429,linear_trend
3,braised beef used,2025-11-01,88976.000000,g,weak,0.131807,-3604.000000,linear_trend
4,braised beef used,2025-12-01,85372.000000,g,weak,0.131807,-3604.000000,linear_trend
5,braised beef used,2026-01-01,81768.000000,g,weak,0.131807,-3604.000000,linear_trend
6,braised chicken,2025-11-01,82508.000000,g,weak,0.016930,-1229.142857,linear_trend
7,braised chicken,2025-12-01,81278.857143,g,weak,0.016930,-1229.142857,linear_trend
8,braised chicken,2026-01-01,80049.714286,g,weak,0.016930,-1229.142857,linear_trend
9,braised pork,2025-11-01,66120.000000,g,weak,0.162311,3268.571429,linear_trend


In [133]:
# ============================================================================
# BLOCK 17: FORECAST ACCURACY METRICS
# ============================================================================

# Calculate summary statistics for forecasts
forecast_summary = forecast_3months.groupby('ingredient').agg({
    'forecasted_usage': ['mean', 'min', 'max'],
    'trend_strength': 'first',
    'r_squared': 'first'
}).reset_index()

forecast_summary.columns = ['ingredient', 'avg_forecasted_usage', 
                            'min_forecast', 'max_forecast', 
                            'trend_strength', 'r_squared']

# Compare with historical average
historical_avg = forecast_data.groupby('ingredient')['total_usage'].mean().reset_index()
historical_avg.columns = ['ingredient', 'historical_avg']

forecast_summary = forecast_summary.merge(historical_avg, on='ingredient', how='left')

# Calculate % change from historical
forecast_summary['pct_change_from_historical'] = (
    (forecast_summary['avg_forecasted_usage'] - forecast_summary['historical_avg']) 
    / forecast_summary['historical_avg'] * 100
)

print("Forecast Summary with Trend Analysis:")
display(forecast_summary.sort_values('pct_change_from_historical', ascending=False).head(10))


Forecast Summary with Trend Analysis:


,ingredient,avg_forecasted_usage,min_forecast,max_forecast,trend_strength,r_squared,historical_avg,pct_change_from_historical
4,carrot,3861.809524,3400.666667,4322.952381,strong,0.747014,1786.666667,116.146055
8,peas,3861.809524,3400.666667,4322.952381,strong,0.747014,1786.666667,116.146055
13,white onion,7723.619048,6801.333333,8645.904762,strong,0.747014,3573.333333,116.146055
11,rice,135163.333333,119023.333333,151303.333333,strong,0.747014,62533.333333,116.146055
3,braised pork,69388.571429,66120.000000,72657.142857,weak,0.162311,54680.000000,26.899363
6,egg,1091.257143,1067.200000,1115.314286,weak,0.048926,983.000000,11.012934
7,green onion,35926.666667,35886.666667,35966.666667,weak,0.000112,35746.666667,0.503543
2,braised chicken,81278.857143,80049.714286,82508.000000,weak,0.016930,86810.000000,-6.371550
10,ramen,1050.780952,1020.695238,1080.866667,weak,0.053216,1186.166667,-11.413718
5,cilantro,28203.047619,27320.761905,29085.333333,weak,0.064758,32173.333333,-12.340300


In [134]:
# ============================================================================
# BLOCK 18: COMBINE HISTORICAL + FORECAST FOR VISUALIZATION
# ============================================================================

# Prepare historical data
historical = forecast_data[['period', 'ingredient', 'total_usage', 'unit']].copy()
historical['data_type'] = 'historical'
historical = historical.rename(columns={'total_usage': 'value'})

# Prepare forecast data
forecast_viz = forecast_3months[['period', 'ingredient', 'forecasted_usage', 'unit']].copy()
forecast_viz['data_type'] = 'forecast'
forecast_viz = forecast_viz.rename(columns={'forecasted_usage': 'value'})

# Combine
combined_timeline = pd.concat([historical, forecast_viz], ignore_index=True)
combined_timeline = combined_timeline.sort_values(['ingredient', 'period'])

print("Combined historical + forecast timeline created")
print(f"Total data points: {len(combined_timeline)}")

# Sample visualization data
sample_ingredient = combined_timeline['ingredient'].value_counts().index[0]
sample_data = combined_timeline[combined_timeline['ingredient'] == sample_ingredient]

print(f"\nSample data for '{sample_ingredient}':")
display(sample_data)


Combined historical + forecast timeline created
Total data points: 126

Sample data for 'boychoy':


,period,ingredient,value,unit,data_type
4,2025-05-01,boychoy,63650.000000,g,historical
17,2025-06-01,boychoy,37150.000000,g,historical
31,2025-07-01,boychoy,40750.000000,g,historical
47,2025-08-01,boychoy,44300.000000,g,historical
61,2025-09-01,boychoy,48900.000000,g,historical
75,2025-10-01,boychoy,46000.000000,g,historical
96,2025-11-01,boychoy,41846.666667,g,forecast
97,2025-12-01,boychoy,40433.809524,g,forecast
98,2026-01-01,boychoy,39020.952381,g,forecast


In [135]:
# ============================================================================
# BLOCK 19: REORDER RECOMMENDATIONS WITH FORECAST
# ============================================================================

# Load existing reorder data
reorder_current = pd.read_csv("processed/analytics_reorder_table.csv")

# Get next month forecast
next_month_forecast = forecast_3months[
    forecast_3months['period'] == forecast_3months['period'].min()
][['ingredient', 'forecasted_usage']].copy()

# Merge with current reorder data
reorder_with_forecast = reorder_current.merge(
    next_month_forecast,
    on='ingredient',
    how='left'
)

# Calculate forecasted weekly usage
reorder_with_forecast['forecasted_weekly_usage'] = (
    reorder_with_forecast['forecasted_usage'] / 4.33
)

# Recalculate days until depletion with forecast
reorder_with_forecast['forecasted_days_until_depletion'] = (
    reorder_with_forecast['avg_quantity_per_shipment_grams'] /
    reorder_with_forecast['forecasted_weekly_usage']
) * 7

# Update alerts based on forecast
def get_forecast_alert(days):
    if pd.isna(days):
        return "⚠️ Unknown (No Forecast Data)"
    elif days < 7:
        return "🔴 Critical - Urgent Reorder"
    elif days < 14:
        return "🟡 Reorder Soon"
    else:
        return "🟢 Sufficient"

reorder_with_forecast['forecasted_alert'] = (
    reorder_with_forecast['forecasted_days_until_depletion'].apply(get_forecast_alert)
)

print("Reorder Recommendations with 3-Month Forecast:")
print("\nAlert Distribution (Forecasted):")
print(reorder_with_forecast['forecasted_alert'].value_counts())

display(reorder_with_forecast[[
    'ingredient', 'total_usage', 'forecasted_usage',
    'weekly_usage_estimate', 'forecasted_weekly_usage',
    'days_until_depletion', 'forecasted_days_until_depletion',
    'reorder_alert', 'forecasted_alert'
]].head(10))


Reorder Recommendations with 3-Month Forecast:

Alert Distribution (Forecasted):
forecasted_alert
🔴 Critical - Urgent Reorder      6
⚠️ Unknown (No Forecast Data)    3
🟢 Sufficient                     3
🟡 Reorder Soon                   2
Name: count, dtype: int64


,ingredient,total_usage,forecasted_usage,weekly_usage_estimate,forecasted_weekly_usage,days_until_depletion,forecasted_days_until_depletion,reorder_alert,forecasted_alert
0,boychoy,46791.666667,41846.666667,10806.389530,9664.357198,NaN,NaN,🔵 Unknown (No Shipment Data),⚠️ Unknown (No Forecast Data)
1,braised beef used,101590.000000,88976.000000,23461.893764,20548.729792,1.804426,2.060237,🔴 Critical - Reorder Now,🔴 Critical - Urgent Reorder
2,braised chicken,86810.000000,82508.000000,20048.498845,19054.965358,3.167463,3.332616,🔴 Critical - Reorder Now,🔴 Critical - Urgent Reorder
3,braised pork,54680.000000,66120.000000,12628.175520,15270.207852,NaN,NaN,🔵 Unknown (No Shipment Data),⚠️ Unknown (No Forecast Data)
4,carrot,1786.666667,3400.666667,412.625096,785.373364,51.299901,26.952310,🟢 Sufficient,🟢 Sufficient
5,cilantro,32173.333333,29085.333333,7430.331024,6717.167052,1.068305,1.181727,🔴 Critical - Reorder Now,🔴 Critical - Urgent Reorder
6,egg,983.000000,1067.200000,227.020785,246.466513,37.001017,34.081709,🟢 Sufficient,🟢 Sufficient
7,green onion,35746.666667,35886.666667,8255.581216,8287.913780,3.846057,3.831053,🔴 Critical - Reorder Now,🔴 Critical - Urgent Reorder
8,peas,1786.666667,3400.666667,412.625096,785.373364,51.299901,26.952310,🟢 Sufficient,🟢 Sufficient
9,pickle cabbage,31916.666667,16456.666667,7371.054657,3800.615858,NaN,NaN,🔵 Unknown (No Shipment Data),⚠️ Unknown (No Forecast Data)


In [136]:
# ============================================================================
# BLOCK 20: SEASONAL TREND ANALYSIS
# ============================================================================

# Extract month from period for seasonal analysis
forecast_data['month'] = pd.to_datetime(forecast_data['period']).dt.month
forecast_data['month_name'] = pd.to_datetime(forecast_data['period']).dt.strftime('%B')

# Calculate average usage by month for each ingredient
seasonal_patterns = (
    forecast_data.groupby(['ingredient', 'month', 'month_name'])['total_usage']
    .mean()
    .reset_index()
    .sort_values(['ingredient', 'month'])
)

# Identify peak and low months for each ingredient
seasonal_summary = []
for ingredient in seasonal_patterns['ingredient'].unique():
    ing_data = seasonal_patterns[seasonal_patterns['ingredient'] == ingredient]
    
    if len(ing_data) >= 2:  # Need at least 2 months for comparison
        peak_month = ing_data.loc[ing_data['total_usage'].idxmax()]
        low_month = ing_data.loc[ing_data['total_usage'].idxmin()]
        
        volatility = ing_data['total_usage'].std()
        avg_usage = ing_data['total_usage'].mean()
        
        seasonal_summary.append({
            'ingredient': ingredient,
            'peak_month': peak_month['month_name'],
            'peak_usage': peak_month['total_usage'],
            'low_month': low_month['month_name'],
            'low_usage': low_month['total_usage'],
            'seasonal_variation_%': ((peak_month['total_usage'] - low_month['total_usage']) 
                                     / low_month['total_usage'] * 100) if low_month['total_usage'] > 0 else 0,
            'volatility': volatility,
            'avg_monthly_usage': avg_usage
        })

seasonal_df = pd.DataFrame(seasonal_summary)
seasonal_df = seasonal_df.sort_values('seasonal_variation_%', ascending=False)

print("Seasonal Trend Analysis:")
print("\nTop 10 Ingredients with Highest Seasonal Variation:")
display(seasonal_df.head(10))

Seasonal Trend Analysis:

Top 10 Ingredients with Highest Seasonal Variation:


,ingredient,peak_month,peak_usage,low_month,low_usage,seasonal_variation_%,volatility,avg_monthly_usage
4,carrot,October,3120.0,July,750.0,316.000000,998.171662,1786.666667
8,peas,October,3120.0,July,750.0,316.000000,998.171662,1786.666667
11,rice,October,109200.0,July,26250.0,316.000000,34936.008167,62533.333333
13,white onion,October,6240.0,July,1500.0,316.000000,1996.343324,3573.333333
9,pickle cabbage,May,52300.0,October,24000.0,117.916667,10389.642278,31916.666667
3,braised pork,September,70360.0,June,34800.0,102.183908,15178.136908,54680.000000
10,ramen,May,1620.0,June,890.0,82.022472,243.991325,1186.166667
2,braised chicken,May,116220.0,June,64760.0,79.462631,17672.743986,86810.000000
5,cilantro,May,43800.0,June,24620.0,77.904143,6486.276795,32173.333333
7,green onion,May,46300.0,June,26160.0,76.987768,7083.204548,35746.666667


In [137]:
# ============================================================================
# BLOCK 21: COST IMPACT ANALYSIS (IF SHIPMENT COST DATA AVAILABLE)
# ============================================================================

# Load shipment data
shipment_data = pd.read_csv("processed/shipments_clean.csv")

# Calculate estimated monthly costs based on forecasted usage
cost_analysis = []

for _, row in forecast_summary.iterrows():
    ingredient = row['ingredient']
    avg_forecast = row['avg_forecasted_usage']
    
    # Find shipment info
    ship_info = shipment_data[shipment_data['ingredient_norm'] == ingredient]
    
    if not ship_info.empty:
        ship_row = ship_info.iloc[0]
        
        # Calculate monthly shipments needed
        monthly_usage = avg_forecast
        quantity_per_shipment = ship_row['quantity_in_grams']
        
        if quantity_per_shipment > 0:
            shipments_needed = np.ceil(monthly_usage / quantity_per_shipment)
        else:
            shipments_needed = 0
        
        cost_analysis.append({
            'ingredient': ingredient,
            'avg_monthly_forecast': avg_forecast,
            'qty_per_shipment_g': quantity_per_shipment,
            'shipments_needed_per_month': shipments_needed,
            'frequency': ship_row['frequency']
        })

cost_df = pd.DataFrame(cost_analysis)

print("Shipment Requirements Based on Forecast:")
display(cost_df.sort_values('shipments_needed_per_month', ascending=False).head(10))


Shipment Requirements Based on Forecast:


,ingredient,avg_monthly_forecast,qty_per_shipment_g,shipments_needed_per_month,frequency
3,cilantro,28203.047619,2267.96,13.0,weekly
8,rice,135163.333333,22679.60,6.0,Biweekly
0,braised beef used,85372.000000,18143.68,5.0,weekly
1,braised chicken,81278.857143,18143.68,5.0,weekly
9,rice noodles,107811.428571,22679.60,5.0,monthly
5,green onion,35926.666667,9071.84,4.0,weekly
2,carrot,3861.809524,9071.84,1.0,weekly
4,egg,1091.257143,6000.00,1.0,weekly
6,peas,3861.809524,9071.84,1.0,weekly
7,ramen,1050.780952,5000.00,1.0,biweekly


In [138]:
# ============================================================================
# BLOCK 22: SAVE ALL FORECAST OUTPUTS
# ============================================================================

# Save main forecast file
forecast_3months.to_csv("processed/demand_forecast_3months.csv", index=False)

# Save combined historical + forecast
combined_timeline.to_csv("processed/historical_demand.csv", index=False)

# Save forecast summary
forecast_summary.to_csv("processed/forecast_summary.csv", index=False)

# Save seasonal analysis
seasonal_df.to_csv("processed/seasonal_trends.csv", index=False)

# Save reorder with forecast
reorder_with_forecast.to_csv("processed/reorder_alerts.csv", index=False)

# Save cost analysis
cost_df.to_csv("processed/cost_drivers.csv", index=False)

print("\n" + "="*70)
print("ALL FORECAST FILES SAVED SUCCESSFULLY!")
print("="*70)
print("\nGenerated Files:")
print("1. demand_forecast_3months.csv - 3-month ingredient demand forecast")
print("2. historical_demand.csv - Combined historical + forecast timeline")
print("3. forecast_summary.csv - Forecast summary with trends")
print("4. seasonal_trends.csv - Seasonal pattern analysis")
print("5. reorder_alerts.csv - Updated reorder recommendations with forecast")
print("6. cost_drivers.csv - Shipment cost projections")
print("="*70)



ALL FORECAST FILES SAVED SUCCESSFULLY!

Generated Files:
1. demand_forecast_3months.csv - 3-month ingredient demand forecast
2. historical_demand.csv - Combined historical + forecast timeline
3. forecast_summary.csv - Forecast summary with trends
4. seasonal_trends.csv - Seasonal pattern analysis
5. reorder_alerts.csv - Updated reorder recommendations with forecast
6. cost_drivers.csv - Shipment cost projections


In [139]:
# ============================================================================
# BLOCK 23: GENERATE FINAL SUMMARY REPORT
# ============================================================================

print("\n" + "="*70)
print("DEMAND FORECAST SUMMARY REPORT")
print("="*70)

print(f"\n Forecast Period: {forecast_3months['period'].min().strftime('%B %Y')} to {forecast_3months['period'].max().strftime('%B %Y')}")
print(f" Total Ingredients Forecasted: {forecast_3months['ingredient'].nunique()}")

print("\n Trend Strength Distribution:")
trend_dist = forecast_3months.groupby('trend_strength')['ingredient'].nunique()
for trend, count in trend_dist.items():
    print(f"   - {trend.capitalize()}: {count} ingredients")

print("\n Top 5 Ingredients by Forecasted Growth:")
top_growth = forecast_summary.nlargest(5, 'pct_change_from_historical')[
    ['ingredient', 'pct_change_from_historical', 'avg_forecasted_usage']
]
for _, row in top_growth.iterrows():
    print(f"   - {row['ingredient']}: +{row['pct_change_from_historical']:.1f}% ({row['avg_forecasted_usage']:.0f} units/month)")

print("\n Top 5 Ingredients by Forecasted Decline:")
top_decline = forecast_summary.nsmallest(5, 'pct_change_from_historical')[
    ['ingredient', 'pct_change_from_historical', 'avg_forecasted_usage']
]
for _, row in top_decline.iterrows():
    print(f"   - {row['ingredient']}: {row['pct_change_from_historical']:.1f}% ({row['avg_forecasted_usage']:.0f} units/month)")

print("\n Critical Reorder Alerts (Next Month):")
critical = reorder_with_forecast[
    reorder_with_forecast['forecasted_alert'].str.contains('Critical|Urgent', na=False)
].sort_values('forecasted_days_until_depletion')

if len(critical) > 0:
    for _, row in critical.head(5).iterrows():
        print(f"   - {row['ingredient']}: {row['forecasted_days_until_depletion']:.1f} days supply remaining")
else:
    print(" No critical alerts - all ingredients sufficiently stocked")

print("\n Seasonal Insights:")
if len(seasonal_df) > 0:
    high_var = seasonal_df.nlargest(3, 'seasonal_variation_%')
    for _, row in high_var.iterrows():
        print(f"   - {row['ingredient']}: {row['seasonal_variation_%']:.0f}% variation (Peak: {row['peak_month']})")

print("\n" + "="*70)
print("FORECAST PIPELINE COMPLETE!")
print("="*70)


DEMAND FORECAST SUMMARY REPORT

 Forecast Period: November 2025 to January 2026
 Total Ingredients Forecasted: 14

 Trend Strength Distribution:
   - Moderate: 1 ingredients
   - Strong: 4 ingredients
   - Weak: 9 ingredients

 Top 5 Ingredients by Forecasted Growth:
   - carrot: +116.1% (3862 units/month)
   - peas: +116.1% (3862 units/month)
   - white onion: +116.1% (7724 units/month)
   - rice: +116.1% (135163 units/month)
   - braised pork: +26.9% (69389 units/month)

 Top 5 Ingredients by Forecasted Decline:
   - pickle cabbage: -62.3% (12040 units/month)
   - braised beef used: -16.0% (85372 units/month)
   - rice noodles: -14.9% (107811 units/month)
   - boychoy: -13.6% (40434 units/month)
   - cilantro: -12.3% (28203 units/month)

 Critical Reorder Alerts (Next Month):
   - cilantro: 1.2 days supply remaining
   - braised beef used: 2.1 days supply remaining
   - rice: 2.9 days supply remaining
   - rice noodles: 3.1 days supply remaining
   - braised chicken: 3.3 days supply